# Custom Aggregations

This notebook is motivated by a
[post](https://discourse.pangeo.io/t/using-xhistogram-to-bin-measurements-at-particular-stations/2365/4)
on the Pangeo discourse forum.

> Even better would be a command that lets me simply do the following.
>
>     A = da.groupby(['lon_bins', 'lat_bins']).mode()

This notebook will describe how to accomplish this using a custom `Aggregation`.


```{tip}
flox now supports `mode`, `nanmode`, `quantile`, `nanquantile`, `median`, `nanmedian` using exactly the same 
approach as shown below
```


In [ ]:
import numpy as np
import numpy_groupies as npg
import xarray as xr

import flox.xarray
from flox import Aggregation
from flox.aggregations import mean

# define latitude and longitude bins
binsize = 1.0  # 1°x1° bins
lon_min, lon_max, lat_min, lat_max = [-180, 180, -65, 65]
lon_bins = np.arange(lon_min, lon_max, binsize)
lat_bins = np.arange(lat_min, lat_max, binsize)

size = 28397


da = xr.DataArray(
    np.random.randint(0, 7, size=size),
    dims="profile",
    coords={
        "lat": (
            "profile",
            (np.random.random(size) - 0.5) * (lat_max - lat_min),
        ),
        "lon": (
            "profile",
            (np.random.random(size) - 0.5) * (lon_max - lon_min),
        ),
    },
    name="label",
)
da

## A built-in reduction

First a simple example of lat-lon binning using a built-in reduction: mean


In [ ]:
binned_mean = flox.xarray.xarray_reduce(
    da,
    da.lat,
    da.lon,
    func="mean",  # built-in
    expected_groups=(lat_bins, lon_bins),
    isbin=(True, True),
)
binned_mean.plot()

## Aggregations

flox knows how to interperet `func="mean"` because it's been implemented in
`aggregations.py` as an
[Aggregation](https://flox.readthedocs.io/en/latest/generated/flox.aggregations.Aggregation.html)

An `Aggregation` is a blueprint for computing an aggregation, with both numpy
and dask data.


In [ ]:
print(type(mean))
mean

Here's how the mean Aggregation is created

```python
mean = Aggregation(
    name="mean",

    # strings in the following are built-in grouped reductions
    # implemented by the underlying  "engine": flox or numpy_groupies or numbagg

    # for pure  numpy inputs
    numpy="mean",

    # The next are for dask inputs and describe how to reduce
    # the data in parallel
    chunk=("sum", "nanlen"), # first compute these blockwise : (grouped_sum, grouped_count)
    combine=("sum", "sum"), #  reduce intermediate results (sum the sums, sum the counts)
    finalize=lambda sum_, count: sum_ / count, # final mean value (divide sum by count)

    fill_value=(0, 0),  # fill value for intermediate  sums and counts when groups have no members
    dtypes=(None, np.intp),  # optional dtypes for intermediates
    final_dtype=np.floating,  # final dtype for output
)
```


## Defining a custom aggregation

First we'll need a function that executes the grouped reduction given numpy
inputs.

Custom functions are required to have this signature (copied form
numpy_groupies):

```python

def custom_grouped_reduction(
    group_idx, array, *, axis=-1, size=None, fill_value=None, dtype=None
):
    """
    Parameters
    ----------

    group_idx : np.ndarray, 1D
        integer codes for group labels (1D)
    array : np.ndarray, nD
        values to reduce (nD)
    axis : int
        axis of array along which to reduce. Requires array.shape[axis] == len(group_idx)
    size : int, optional
        expected number of groups. If none, output.shape[-1] == number of uniques in group_idx
    fill_value : optional
        fill_value for when number groups in group_idx is less than size
    dtype : optional
        dtype of output

    Returns
    -------

    np.ndarray with array.shape[-1] == size, containing a single value per group
    """
    pass
```

Since numpy_groupies does not implement a median, we'll do it ourselves by
passing `np.median` to `numpy_groupies.aggregate_numpy.aggregate`. This will
loop over all groups, and then execute `np.median` on the group members in
serial. It is not fast, but quite convenient.


In [ ]:
def grouped_median(group_idx, array, *, axis=-1, size=None, fill_value=None, dtype=None):
    return npg.aggregate_numpy.aggregate(
        group_idx,
        array,
        func=np.median,
        axis=axis,
        size=size,
        fill_value=fill_value,
        dtype=dtype,
    )

Now we create the `Aggregation`


In [ ]:
agg_median = Aggregation(
    name="median",
    numpy=grouped_median,
    fill_value=-1,
    chunk=None,
    combine=None,
)
agg_median

And apply it!


In [ ]:
flox.xarray.xarray_reduce(
    da,
    da.lat,
    da.lon,
    func=agg_median,
    expected_groups=(lat_bins, lon_bins),
    isbin=(True, True),
    fill_value=np.nan,
)